# Data Engineering - Lab 1


## Intro to table of contents

In this lab we'll be introduced to the Google Colab environment. To start this notebook simply go to "Runtime" and choose "Run All". This will connect Colab to your virtual machine and run the code needed to get your database setup.

You'll notice that notebooks are setup with a table of contents navigation bar on the left nav bar of your screen. Mouse over and hover to see which options are available and click "Table of Contents" to see the major sections to this Notebook.  

## Database setups (do not modify)

In [ ]:
# setups including getting the database

!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

!wget -O northwind.db https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db

import sqlite3
con = sqlite3.connect("northwind.db")

%load_ext sql
%sql sqlite:///northwind.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.27.0 which is incompatible.
--2024-09-03 16:04:42--  https://github.com/matthewpecsok/data_engineering/raw/main/data/northwind.db
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db [following]
--2024-09-03 16:04:42--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/northwind.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.1

# Mount Google Drive and Export Your Works

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

## ERD of the Northwind Database


![ERD](https://github.com/matthewpecsok/data_engineering/blob/main/data/Northwind_ERD.png?raw=true)


Using %sql allows for one line sql statements
Using %%sql allows for multi-line sql statements and better readability for long sql queries.

In [ ]:
%sql select * from products limit 4

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22,53,0,0,0


In [ ]:
%%sql

select
count(1) as product_count
from
products

,product_count
0,77


## Lab question 1

In [ ]:
%%sql
select count(1) as unitprice_gt_18
from Products where UnitPrice > 18

,unitprice_gt_18
0,43


## Lab question 2


In [ ]:
%%sql select
c.CompanyName,
count(o.OrderID) as order_count


from Orders o
join Customers c on o.CustomerID = c.CustomerID

group by c.CompanyName
order by order_count
desc
limit 5

,CompanyName,order_count
0,Save-a-lot Markets,31
1,Ernst Handel,30
2,QUICK-Stop,28
3,Hungry Owl All-Night Grocers,19
4,Folk och fä HB,19


## Lab question 3



In [ ]:
%%sql select
od.OrderID,
ca.CategoryName

from Products p
join "Order Details" od on od.ProductID = p.ProductID
join Categories ca on ca.CategoryID = p.CategoryID

order by ca.CategoryID
limit 5

,OrderID,CategoryName
0,10253,Beverages
1,10254,Beverages
2,10255,Beverages
3,10257,Beverages
4,10258,Beverages


## Lab question 4



In [ ]:
%%sql
select
    COUNT(OrderID) AS RegionCount,
    ShipRegion AS ShipRegion
from
    Orders
group by
    ShipRegion
having
    COUNT(OrderID) > 100
order by
    RegionCount desc

,RegionCount,ShipRegion
0,276,Western Europe
1,152,North America
2,145,South America


## Lab Question 5


In [ ]:
%%sql
select
    max(UnitPrice) as max_unit_price,
    min(UnitPrice) as min_unit_price,
    round(avg(UnitPrice), 2) as average_unit_price
from
    Products

,max_unit_price,min_unit_price,average_unit_price
0,263.5,2.5,28.87


## Lab Question 6

In [ ]:
%%sql
select
    (od.UnitPrice * od.Quantity) as original_total,
    (od.UnitPrice * od.Quantity) - (od.UnitPrice * od.Quantity * od.Discount) as discounted_price,
    (od.UnitPrice * od.Quantity * od.Discount) as discount_amount,
    (od.Discount) as discount_percentage,
    c.CompanyName
from
    "Order Details" od
join
    Orders o on od.OrderID = o.OrderID
join
    Customers c on o.CustomerID = c.CustomerID
where
    c.CompanyName = 'Ernst Handel'
    and (od.UnitPrice * od.Quantity) > 4000;

,original_total,discounted_price,discount_amount,discount_percentage,CompanyName
0,4216.0,4005.2,210.8,0.05,Ernst Handel
1,6360.0,6042.0,318.0,0.05,Ernst Handel
2,6050.0,6050.0,0.0,0.00,Ernst Handel
3,4322.5,4322.5,0.0,0.00,Ernst Handel


## Lab Question 7

In [ ]:
%%sql
select
    strftime('%Y', OrderDate) AS order_year,
    COUNT(OrderID) AS order_count
from
    Orders
group by
    order_year
order by
    order_year;

,order_year,order_count
0,2016,152
1,2017,408
2,2018,270


## Lab Question 8

In [ ]:
%%sql
select
    e.EmployeeID,
    e.LastName,
    e.FirstName,
    count(o.OrderID) as order_count
from
    Employees e
join
    Orders o ON e.EmployeeID = o.EmployeeID
group by
    e.EmployeeID, e.LastName, e.FirstName
order by
    order_count desc
limit 5

,EmployeeID,LastName,FirstName,order_count
0,4,Peacock,Margaret,156
1,3,Leverling,Janet,127
2,1,Davolio,Nancy,123
3,8,Callahan,Laura,104
4,2,Fuller,Andrew,96


## Lab Question 9

In [ ]:
%%sql
select
    COUNT(*)
from
    sqlite_master
where
    type = 'table';

,COUNT(*)
0,14


## Lab Question 10

In [ ]:
%%sql
select count(*) as view_count
from sqlite_master
where type = 'view'

,view_count
0,17


## Lab Question 11

In [ ]:
%%sql
SELECT
    COUNT(*) AS left_count
FROM
    Customers c
LEFT JOIN
    Orders o ON c.CustomerID = o.CustomerID;

,left_count
0,834


## Lab Question 12

In [ ]:
%%sql
SELECT
    COUNT(*) AS inner_count
FROM
    Customers c
INNER JOIN
    Orders o ON c.CustomerID = o.CustomerID;

,inner_count
0,830


## Lab Question 13

In [ ]:
%sql create table CustomerFavoriteOrders (CustomerID_id integer, Orders_id integer,Customers_score integer)

""


## Lab Question 14

In [ ]:
%sql insert into CustomerFavoriteOrders values (1,1,10)
%sql insert into CustomerFavoriteOrders values (1,2,7.5)
%sql insert into CustomerFavoriteOrders values (1,3,7.5)
%sql insert into CustomerFavoriteOrders values (1,5,3.5)
%sql insert into CustomerFavoriteOrders values (1,9,5.5)

""


## Lab Question 15

In [ ]:
%sql select * from CustomerFavoriteOrders

,CustomerID_id,Orders_id,Customers_score
0,1,1,10.0
1,1,2,7.5
2,1,3,7.5
3,1,5,3.5
4,1,9,5.5


add additional sections as needed to complete the lab

In [ ]:
# replace ###### with your file name
# make sure you have your google drive mounted.

!cp "/content/drive/MyDrive/Colab Notebooks/A1_Vu_Nguyen.ipynb" ./
!jupyter nbconvert --to html "A1_Vu_Nguyen.ipynb"

[NbConvertApp] Converting notebook A1_Vu_Nguyen.ipynb to html
[NbConvertApp] Writing 712468 bytes to A1_Vu_Nguyen.html
